In [86]:
#Iniciando modulos
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.select import Select
import pandas as pd
import pyodbc
import time
import re
import os
import csv

In [87]:
def executar_conexao_bd():
    server = 'CNBDL-GLEISSON'
    database = 'Buscar_material'
    #username = 'gleisson.santos'
    #password = '12345'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)#+ ';UID=' + username + ';PWD=' + password)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

Executado a conexão com o BD......


In [9]:
#Acesso ao banco de dados Casa
def executar_conexao_bd():
    server = 'DESKTOP-0P0H9IO'
    database = 'Dados_Pavimento'
    conexao = pyodbc.connect(
        'DRIVER={SQL Server};SERVER=' + server + ';DATABASE=' + database)
    print('Executado a conexão com o BD......')
    return conexao

cursor = executar_conexao_bd()
cursor

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server inexistente ou acesso negado. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53)')

In [89]:
options = Options()
options.add_argument("start-maximized")
driver = webdriver.Chrome(ChromeDriverManager().install())
chrome_options = webdriver.ChromeOptions()

AttributeError: 'str' object has no attribute 'capabilities'

In [90]:
driver = webdriver.Chrome()

In [91]:
login = 't034183'
senha = 'CNB@2023*'
driver.get("https://sciweb.embasanet.ba.gov.br/sci-web/")
time.sleep(2)

# Procurar a tag random do site / Login e senha
randomtag = driver.find_element(By.ID, "random-tag").get_attribute('value')
driver.find_element(By.ID, "loginForm-usuario-{randomtag}".format(randomtag=randomtag)).send_keys(login)
driver.find_element(By.ID, "loginForm-senha-{randomtag}".format(randomtag=randomtag)).send_keys(senha)
driver.find_element(By.ID, "loginForm-submit").click()
time.sleep(2)

In [92]:
#tipo: insu, menu: GOSI_anchor - ACESSAR MENU DE INSUMOS
def arvore_acesso(driver, tipo, menu ):
    # Proc detalhe da Os no campo de buscar
    driver.find_element(By.ID, "arvoreSearch").send_keys(tipo)
    # aguardar ate o elemento aparecer e/ou seguir codigo
    wait = WebDriverWait(driver, 5)
    element1 = wait.until(EC.element_to_be_clickable((By.ID, menu)))
    # acessar Campo detalhe da Os
    driver.find_element(By.ID, menu).click()
    time.sleep(3)
    #acessar camada frame do site
    driver.switch_to.frame("frame-content")

#ACESSAR MENU PAR EXTRAR INSUMADOS
arvore_acesso(driver, 'detalhe', 'DOSE_anchor')

####  Acesso ao item "Detalhes da Os"

In [93]:
#Consultar na label Material
def preencher_ss(driver, lista, lista2):
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').clear()
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt71-item"]').send_keys(lista)
    time.sleep(0.5)
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-j_idt79-item"]').send_keys(lista2)
    time.sleep(0.5)
    # clinical em buscar
    driver.find_element(By.XPATH, '//*[@id="form-j_idt70-panel"]/input').click()
    #coletar valor solicitado
    
    try:
        material = driver.find_element(By.CSS_SELECTOR, "label[title='Material Utilizado'] ").text
    except NoSuchElementException:
        material = 'None'
    
    try:
        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD'] ").text
    except NoSuchElementException:
        try:
            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do HD Instalado'] ").text
        except NoSuchElementException:
            try:
                hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro Instalado'] ").text
            except NoSuchElementException:
                try:
                    hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro'] ").text
                except NoSuchElementException:
                    try:
                        hd = driver.find_element(By.CSS_SELECTOR, "label[title='Hidrômetro Instalado'] ").text
                    except NoSuchElementException:
                        try:
                            hd = driver.find_element(By.CSS_SELECTOR, "label[title='Número do Hidrômetro para Religação'] ").text
                        except NoSuchElementException:
                            hd = None
                        
    return material, hd


In [94]:
df = pd.read_sql('SELECT TOP 10 * FROM [HDs]', executar_conexao_bd())
#df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço', 'Material', 'Mureta','Hd'])
df = pd.DataFrame(df, columns = ['SS', 'OS','Serviço','Hd','Material','Mureta'])
#df = df.astype(str)

#df = df.query("Tipo == '21 - LIGACAO DE AGUA'")

display(df)
n_linhas = len(df)
print("Número de linhas: ", n_linhas)

Executado a conexão com o BD......


C:\Users\gleisson.santos.TECBRI\AppData\Local\Temp\ipykernel_19048\2393116317.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql('SELECT TOP 10 * FROM [HDs]', executar_conexao_bd())


,SS,OS,Serviço,Hd,Material,Mureta
0,994706454,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557185,None,None
1,995227682,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557027,None,None
2,994549002,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557116,None,None
3,995329243,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557187,None,None
4,995335345,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557058,None,None
5,995336434,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557056,None,None
6,995337390,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557124,None,None
7,986064702,1,194 - SUBST HD VOLUMETRICO,A23L332069,None,None
8,986064741,1,194 - SUBST HD VOLUMETRICO,A23L332075,None,None
9,987246821,1,194 - SUBST HD VOLUMETRICO,A23L332070,None,None


Número de linhas:  10


In [82]:
#Extrair valores e criar coluna Material insumado
dados = []
hds = []
for i, row in df.iterrows():
    a = preencher_ss(driver, row['SS'], row['OS'])
    print(a)
    time.sleep(0.3)
    dados.append(a[0])
    hds.append(a[1])

df['Material'] = dados
df['Hd'] = hds
df

('', 'Y23SG0557185')
('', 'Y23SG0557027')
('', 'Y23SG0557116')
('', 'Y23SG0557187')
('', 'Y23SG0557058')
('', 'Y23SG0557056')
('', 'Y23SG0557124')
('', 'A23L332069')
('', 'A23L332075')
('', 'A23L332070')


,SS,OS,Serviço,Hd,Material,Mureta
0,994706454,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557185,,None
1,995227682,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557027,,None
2,994549002,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557116,,None
3,995329243,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557187,,None
4,995335345,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557058,,None
5,995336434,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557056,,None
6,995337390,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557124,,None
7,986064702,1,194 - SUBST HD VOLUMETRICO,A23L332069,,None
8,986064741,1,194 - SUBST HD VOLUMETRICO,A23L332075,,None
9,987246821,1,194 - SUBST HD VOLUMETRICO,A23L332070,,None


In [83]:
#Acessar tabela do BD e extrair apenas item do texto
m_encontrado = ['Mureta', 'MURETA','mureta']

material = []

for i, row in df.iterrows():
    if any(x in row['Material'] for x in m_encontrado):
        material.append('Sim')
    else:
        material.append('Não')     
df['Mureta'] = material


#df = df.astype(str)
#df = pd.DataFrame(df, columns = ['SS','OS', 'Serviço', 'Hd', 'Material','Mureta'])
#df_sim = df.loc[df['Mureta'] == 'Não']
#display(df_sim)
df

,SS,OS,Serviço,Hd,Material,Mureta
0,994706454,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557185,,Não
1,995227682,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557027,,Não
2,994549002,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557116,,Não
3,995329243,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557187,,Não
4,995335345,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557058,,Não
5,995336434,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557056,,Não
6,995337390,2,2 - LIG.AGUA DIAM.1/2 C/HID.,Y23SG0557124,,Não
7,986064702,1,194 - SUBST HD VOLUMETRICO,A23L332069,,Não
8,986064741,1,194 - SUBST HD VOLUMETRICO,A23L332075,,Não
9,987246821,1,194 - SUBST HD VOLUMETRICO,A23L332070,,Não


In [85]:
for index, row in df.iterrows():    
    ID = row.Material
    SS = row.SS
    MT = row.Mureta
    HD = row.Hd
    #comando_sql = (f"UPDATE [Janeiro26/12-25/01] SET Material='{ID}', Mureta='{MT}', Hd='{HD}' WHERE SS={SS} ")
    #comando_sql = "UPDATE [700_Lauro] SET Material='{}', Mureta='{}', Hd='{}' WHERE SS={}".format(
    #ID.replace("'", "''"), MT.replace("'", "''"), HD.replace("'", "''"), SS)  
    comando_sql = "UPDATE [HDs] SET Hd='{}' WHERE SS={}".format(
    HD.replace("'", "''"), SS)  
    cursor.execute(comando_sql)
    
cursor.commit()

In [2]:
#Limpar colunas do Banco de dados;
#delete_query = f"DELETE FROM {table_name}"
#df = df.astype(str)
#df = pd.DataFrame(df, columns=['SS', 'OS', 'Matutilizado', 'Hdutilizado', 'Usoumureta'])

#cursor = executar_conexao_bd().cursor()
#for index, row in df.iterrows():
    #Mat = row.Matutilizado
    #HD  = row.Hdutilizado
    #INS = row.Usoumureta
    #SS  = row.SS
    #comando_sql = (f"UPDATE Material SET Matutilizado='NULL', Hdutilizado='NULL', Usoumureta='NULL' WHERE SS ={SS}")
    #cursor.execute(comando_sql)
    
#cursor.commit()
#cursor.close()